In [35]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

import pandas as pd
import json
from datetime import datetime

from riotwatcher import LolWatcher, ApiError

watcher = LolWatcher(config['riotwatcher_api_key'])

In [51]:
response = watcher.match.by_id(region="euw1",match_id="EUW1_6158830217")
# response

In [52]:
response = watcher.match.timeline_by_match(region="euw1",match_id="EUW1_6158830217")

In [53]:
df_events = (
    pd.json_normalize(
        data=response['info']['frames'],
        record_path='events',
    )
)

df_events['timestamp_min'] = pd.to_timedelta(df_events['timestamp'], unit='ms')
df_events.columns

Index(['realTimestamp', 'timestamp', 'type', 'itemId', 'participantId',
       'levelUpType', 'skillSlot', 'creatorId', 'wardType', 'level', 'bounty',
       'killStreakLength', 'killerId', 'shutdownBounty', 'victimDamageDealt',
       'victimDamageReceived', 'victimId', 'position.x', 'position.y',
       'afterId', 'beforeId', 'goldGain', 'assistingParticipantIds',
       'killType', 'laneType', 'teamId', 'killerTeamId', 'monsterSubType',
       'monsterType', 'buildingType', 'towerType', 'actualStartTime',
       'multiKillLength', 'gameId', 'winningTeam', 'timestamp_min'],
      dtype='object')

In [56]:
df_events.type.unique()

array(['PAUSE_END', 'ITEM_PURCHASED', 'SKILL_LEVEL_UP', 'WARD_PLACED',
       'ITEM_DESTROYED', 'LEVEL_UP', 'CHAMPION_KILL', 'ITEM_UNDO',
       'CHAMPION_SPECIAL_KILL', 'WARD_KILL', 'TURRET_PLATE_DESTROYED',
       'ITEM_SOLD', 'ELITE_MONSTER_KILL', 'BUILDING_KILL',
       'OBJECTIVE_BOUNTY_PRESTART', 'GAME_END'], dtype=object)

In [64]:
data_dragon_versions = watcher.data_dragon.versions_for_region(region="euw")
items_data_dragon_version = data_dragon_versions['n']['item']
items_data_dragon_version

current_item_list = watcher.data_dragon.items(items_data_dragon_version)

In [122]:
dict_current_item_list = {
    "item_id": [],
    "item_name": [],
}

for item_id in current_item_list['data'].keys():
    tmp_df_item = pd.json_normalize(data=current_item_list['data'][item_id])

    dict_current_item_list["item_id"].append(item_id)
    dict_current_item_list["item_name"].append(tmp_df_item['name'].to_string(index=False))

df_current_item_list = pd.DataFrame.from_dict(data=dict_current_item_list, orient="columns")
df_current_item_list

,item_id,item_name
0,1001,Boots
1,1004,Faerie Charm
2,1006,Rejuvenation Bead
3,1011,Giant's Belt
4,1018,Cloak of Agility
...,...,...
261,7027,Primordial Dawn
262,7028,Infinite Convergence
263,7050,Gangplank Placeholder
264,8001,Anathema's Chains


In [60]:
df_item_purchased = df_events.query("type == 'ITEM_PURCHASED'").dropna(axis=1)
df_item_purchased['item_id'] = df_item_purchased.itemId
df_item_purchased['item_name'] = []





,timestamp,type,itemId,participantId,timestamp_min
1,2980,ITEM_PURCHASED,1054.0,1.0,0 days 00:00:02.980000
2,2980,ITEM_PURCHASED,2003.0,1.0,0 days 00:00:02.980000
3,3739,ITEM_PURCHASED,3340.0,1.0,0 days 00:00:03.739000
4,4169,ITEM_PURCHASED,1056.0,8.0,0 days 00:00:04.169000
5,4169,ITEM_PURCHASED,2003.0,8.0,0 days 00:00:04.169000
...,...,...,...,...,...
548,1099209,ITEM_PURCHASED,1037.0,2.0,0 days 00:18:19.209000
559,1130752,ITEM_PURCHASED,3191.0,8.0,0 days 00:18:50.752000
561,1132205,ITEM_PURCHASED,2420.0,8.0,0 days 00:18:52.205000
562,1135244,ITEM_PURCHASED,1018.0,3.0,0 days 00:18:55.244000
